<a href="https://colab.research.google.com/github/xiaoya27/Aerial-Image-Recognition/blob/main/MobileNet_Aerial_Image_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aerial imagery Project

<div>
    <h2>Introduction </h2>
     <br>
Aerial imagery has been a primary source of geographic data for quite a long time. With technology progress, aerial imagery became really practical for remote sensing : the science of obtaining information about an object, area or phenomenon.
Nowadays, there are many uses of image recognition spanning from robotics/drone vision to autonomous driving vehicules or face detection.
<br>
In this challenge, we will use pre-processed data, coming from landscape images. The goal is to learn to differentiate common and uncommon landscapes such as a beach, a lake or a meadow.
    Data comes from part of the data set (NWPU-RESISC45) originally used in <a href="https://arxiv.org/pdf/1703.00121.pdf?fbclid=IwAR16qo-EX_Z05ZpxvWG8F-oBU0SlnY-3BPCWBVVOGPyJcVy7BBqCKjnsvJo">Remote Sensing Image Scene Classification</a>. This data set contains 45 categories while we only kept 13 out of them.

References and credits: 
Yuliya Tarabalka, Guillaume Charpiat, Nicolas Girard for the data sets presentation.<br>
Gong Cheng, Junwei Han, and Xiaoqiang Lu, for the original article on the chosen data set.
</div>

In [ ]:
scene_class = ['beach', 'chaparral','cloud','desert','forest','island','lake','meadow','mountain','river','sea','snowberg','wetland']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Step1 Install Requirements 


In [ ]:
!pip install neptune-client==0.4.130

import neptune
from google.colab import output
output.clear() 

In [ ]:
%cd /content/drive/MyDrive/workingfolder/2021/starting_kit/

/content/drive/MyDrive/workingfolder/2021/starting_kit


In [ ]:
neptune.init(
    api_token="xx",
    project_qualified_name="xiaoya27/AerialImageClassification"
)

Project(xiaoya27/AerialImageClassification)

In [ ]:
import numpy as np
import random
import re
from matplotlib import pyplot as plt

In [ ]:
model_dir = "sample_code_submission"
result_dir = 'sample_result_submission/' 
problem_dir = 'ingestion_program/'  
score_dir = 'scoring_program/'
from sys import path; path.append(model_dir); path.append(problem_dir); path.append(score_dir)

In [ ]:
def data_loading():
  #data_dir = 'sample_data'
  data_dir = '../public_data' # download "public_data" from the challenge website
  data_name = 'Areal'
  from data_manager import DataManager
  D = DataManager(data_name, data_dir, replace_missing=False, verbose=True)
  print(D)
  X_train = D.data['X_train']
  Y_train = D.data['Y_train']
  X_valid = D.data['X_valid']
  X_test = D.data['X_test']
  with open('data.npy', 'wb') as f:
      np.save(f, X_train)
      np.save(f, Y_train)
      np.save(f, X_valid )
      np.save(f, X_test)

In [ ]:
# run it once
#data_loading()

In [ ]:
with open('data.npy', 'rb') as f:
    X_train = np.load(f)
    Y_train = np.load(f)
    X_valid = np.load(f)
    X_test = np.load(f)

(5200, 1)

In [ ]:
val_percent = 0.05
train_size=int((1-val_percent)*len(X_train))

In [ ]:
#data_dir = 'sample_data'
data_dir = '../public_data' # download "public_data" from the challenge website
data_name = 'Areal'

In [ ]:

params={}
params['batch_size']=16
params['val_percent']=val_percent



### Step 2 : Building training dataset

In [ ]:
#X_train.reshape((5200,128,128,3))

%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

from tensorflow.keras.applications import MobileNetV2 as MobileNet

feature_extractor_name='MobileNet'

Tensorflow version 2.4.0


In [ ]:
# check AUTOTUNE in https://www.tensorflow.org/guide/data_performance

In [ ]:
#View images from the dataset

def view_image(ds):
    image, label = next(iter(ds)) # extract 1 batch from the dataset
    image = image.numpy()
    label = label.numpy()

    fig = plt.figure(figsize=(22,11))
    for i in range(8):
        ax = fig.add_subplot(2, 4, i+1, xticks=[], yticks=[])
        ax.imshow(image[i])
        ax.set_title(f"Label: {label[i]}")

In [ ]:
#view_image(dataset)

In [ ]:
def process_image(image, label):
    # cast and normalize image
    image = tf.reshape(image, (128,128,3))
    image = tf.image.resize(image, [224, 224],method='nearest')
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.random_flip_left_right(image)
    image = tf.keras.applications.mobilenet.preprocess_input(image)
    return image, label


dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(buffer_size=1024)


In [ ]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

train_dataset = dataset.map(process_image).batch(params['batch_size']).cache().prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = dataset.map(process_image).batch(params['batch_size']).cache().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
from tensorflow.keras.callbacks import Callback
class NeptuneLogger(Callback):

    def on_batch_end(self, batch, logs={}):
        for log_name, log_value in logs.items():
            neptune.log_metric(f'batch_{log_name}', log_value)

    def on_epoch_end(self, epoch, logs={}):
        for log_name, log_value in logs.items():
            neptune.log_metric(f'epoch_{log_name}', log_value)

neptune.create_experiment(name='Aerial-image',
                          params=params,
                          tags=['Tensorflow',feature_extractor_name],
                          )

https://ui.neptune.ai/xiaoya27/AerialImageClassification/e/AER-5


Experiment(AER-5)

### Step 3 . Building&training model

In [ ]:
feature_extractor = MobileNet(input_shape=(224,224,3),
                                          include_top=False,
                                          weights='imagenet')
# what is input shape of backend model
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
                                          layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
                                          layers.experimental.preprocessing.RandomRotation(0.2),])
input = tf.keras.Input(shape=(224,224,3))
x = data_augmentation(input)
x = feature_extractor(input,training=True)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dense(512, activation="relu")(x)
x = tf.keras.layers.Dense(13, activation="softmax", name="classification")(x)                                  

checkpoint_path = "training_1/cp.ckpt"

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

with tf.device('/device:GPU:0'):
  model = tf.keras.Model(inputs=input, outputs=x)

  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy', run_eagerly=True)
  model.fit(train_dataset,
            epochs=15,
            validation_data=val_dataset,
            callbacks=[cp_callback,NeptuneLogger()]
            )
# TBC https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/transfer_learning.ipynb#scrollTo=DgzQX6Veb2WT

Epoch 1/15
325/325 [==============================] - 83s 249ms/step - loss: 1.5463 - accuracy: 0.5367 - val_loss: 0.7686 - val_accuracy: 0.7673

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/15
325/325 [==============================] - 67s 205ms/step - loss: 0.8798 - accuracy: 0.7315 - val_loss: 0.6921 - val_accuracy: 0.7925

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/15
325/325 [==============================] - 67s 206ms/step - loss: 0.6606 - accuracy: 0.8012 - val_loss: 0.5738 - val_accuracy: 0.8246

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/15
325/325 [==============================] - 66s 204ms/step - loss: 0.5347 - accuracy: 0.8357 - val_loss: 0.5015 - val_accuracy: 0.8494

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/15
325/325 [==============================] - 66s 205ms/step - loss: 0.4734 - accuracy: 0.8475 - val_loss: 0.5337 - val_accuracy: 0.8490

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/15
325/325 [==========

In [ ]:
neptune.stop()